### Trade objective
A huge school of goldfish arrived at our shores. And they all brought scuba gear they want to get rid of fast. For the right price of course.
You only have two chances to offer a good price. Each one of the goldfish will accept the lowest bid that is over their reserve price. There’s a constant desire for scuba gear on the archipelago. So, at the end of the round, you’ll be able to sell them for 1000 SeaShells a piece. Your goal is to set prices that ensure a profitable trade with as many goldfish as possible.
Once you are satisfied with your strategy and input, use the ‘Submit manual trade’ button to lock it in. Note that you can (re)submit new strategies as long as the round is still in progress. As soon as the round ends, the trading strategy that was submitted last will be processed.
Good luck and have fun trading!

Set your prices to buy scuba gear. You can set a lowest and a highest bid. Whilst not every goldfish has the same reserve price, you know the distribution of their reserve prices. The reserve price will be no lower than 900 and no higher than 1000. The probability scales linearly from least likely at 900 to most likely at 1000.

In [44]:
# In the prompt, it is given that the reserve price is unknown but
# grows linearly from 900 to 1000.

# I assume that the pdf of the reserve price is parametrized by a (0 < a < 1)
# and it is given by
# f(x) = a/100 + (1-a)*(x-900)/5000

# Let us denote the low bid as x1+900 and the high bid as x2+900
# Doing the integral, the expected reward is given by,

def reward_fun(a,x1,x2):
    result = 0
    result += a * (100 - x1) * x1 / 100.0 + a * (100 - x2) * (x2 - x1) / 100.0
    result += (1 - a) * (100 - x1) * (x1 ** 2) / 10000.0 + (1 - a) * (100 - x2) * (x2 ** 2 - x1 ** 2) / 10000.0
    return result

# Brute force the optimal solutions
def find_optimal(a):
    optim_pairs = 0
    optim_rewards = 0
    for x1 in range(0,101):
        for x2 in range(x1+1, 101):
            curr_reward = reward_fun(a, x1,x2)
            if curr_reward > optim_rewards:
                optim_rewards = curr_reward
                optim_pairs = (x1,x2)
    return optim_rewards, optim_pairs

In [49]:
import statistics as stat

d = {}

for a in range(0, 10000):
    reward, bids  = find_optimal(a / 10000)
    d[a] = (reward, bids)

low_bids = [v[1][0] for v in d.values()]
high_bids = [v[1][1] for v in d.values()]
rewards = [v[0] for v in d.values()]

print('low:', stat.fmean(low_bids, weights=rewards) + 900)
print('high:', stat.fmean(high_bids, weights=rewards) + 900)

print('low:', stat.fmean(low_bids) + 900)
print('high:', stat.fmean(high_bids) + 900)

low: 940.7649430267438
high: 971.9748039242704
low: 941.5358
high: 972.456


In [48]:
# After some clarifications p(900) = 0, which makes a = 0

a = 0

reward, bids  = find_optimal(float(a))
print("reward : ", reward)
print("(low bid, high bid) : ", bids[0] + 900, bids[1] + 900)

reward :  20.4152
(low bid, high bid) :  952 978


In [59]:
import plotly.graph_objs as go
import numpy as np

x = np.linspace(0, 100, 100)
y = np.linspace(0, 100, 100)
X, Y = np.meshgrid(x, y)

# Compute the function values at each point on the grid
Z = reward_fun(0, X, Y)

surface = go.Surface(x=X+900, y=Y+900, z=Z)

# Layout
layout = go.Layout(
    title='Interactive 3D Plot',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y'),
        zaxis=dict(title='Function Value')
    ),
    width=800,
    height=600
)

# Plot figure
fig = go.Figure(data=[surface], layout=layout)

# Show plot
fig.show()